In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
test_set = pd.read_csv("../data/ml-100K/testing_set_seed123_ml100k.csv",sep=",",
    header=None,
    names=["userID", "itemID", "Rating"],)
test_set


In [ ]:
all_users_tp = test_set.groupby('userID')['itemID'].apply(list).tolist()
all_users_tp.__len__()

In [4]:

mf = "model_reco/mf/reco_matrix_mf_100k_100.npy"
neumf = "model_reco/neumf/reco_matrix_neumf_100k_100.npy"
wmf = "model_reco/wmf/reco_matrix_wmf_100k_100.npy"
vaecf ="model_reco/vaecf/reco_matrix_vaecf_ml100k_100.npy"
reco_matrix = np.load(f"../{neumf}")

In [6]:
reco_matrix = reco_matrix[0] # if reco matrix is inside a nested array if not use it directly

In [ ]:

from cornac.metrics import RMSE, AUC, NDCG, Precision, Recall,HitRatio

hr=HitRatio(20)
alist=[]
sum_hr=0
for u in range(reco_matrix.shape[0]):
    x=hr.compute(all_users_tp[u],reco_matrix[u][:20])
    sum_hr=sum_hr+x
plain_hr=sum_hr/reco_matrix.shape[0]

plain_hr


In [ ]:

ndcg=NDCG(20)
alist=[]
sum_ndcg=0
for u in range(reco_matrix.shape[0]):
    x=ndcg.compute(all_users_tp[u],reco_matrix[u][:20])
    sum_ndcg=sum_ndcg+x
plain_ndcg=sum_ndcg/reco_matrix.shape[0]

plain_ndcg

In [10]:
all_pds =[]
reco_matrix_fair=[]
sens="gender"
model="neumf"
# here we load the recommendations over different vaues of beta
all_pds.append(pd.read_csv(f"../results/{model}/{sens}_1.csv",sep=",",))
all_pds.append(pd.read_csv(f"../results/{model}/{sens}_2.csv",sep=",",))
all_pds.append(pd.read_csv(f"../results/{model}/{sens}_3.csv",sep=",",))
all_pds.append(pd.read_csv(f"../results/{model}/{sens}_4.csv",sep=",",))
all_pds.append(pd.read_csv(f"../results/{model}/{sens}_5.csv",sep=",",))
all_pds.append(pd.read_csv(f"../results/{model}/{sens}_6.csv",sep=",",))
all_pds.append(pd.read_csv(f"../results/{model}/{sens}_7.csv",sep=",",))
all_pds.append(pd.read_csv(f"../results/{model}/{sens}_8.csv",sep=",",))
# results/reco_mat_test.npy
for j in range(len(all_pds)):
    reco_m =[]
    for i in range(all_pds[j].__len__()):
        reco_m.append(np.array(all_pds[j].iloc[i][1:]))
    reco_matrix_fair.append(reco_m)
    



In [ ]:

hr=HitRatio(20)
hrlist=[]
for r in reco_matrix_fair:
    sum_hr=0
    for u in range(reco_matrix.shape[0]):
        x=hr.compute(all_users_tp[u],r[u])
        sum_hr=sum_hr+x
    hr_occ=sum_hr/reco_matrix.shape[0]
    hrlist.append(hr_occ)
    

[plain_hr]+hrlist

In [ ]:


hr=NDCG(20)
hrlist=[]
for r in reco_matrix_fair:
    sum_hr=0
    for u in range(reco_matrix.shape[0]):
        x=hr.compute(all_users_tp[u],r[u])
        sum_hr=sum_hr+x
    hr_occ=sum_hr/reco_matrix.shape[0]
    hrlist.append(hr_occ)
    

[plain_ndcg]+hrlist


In [ ]:
movies = pd.read_csv(
    "../data/ml-100K/i_id_mapping_genre.csv",
    sep="\t",
    names=[
        "item_id",
        "Name",
        "genres",
        "itemID"
    ],
    header=0,
    # encoding="latin-1",
)
movies = movies.drop(columns=["item_id"])
movies = movies.sort_values(by="itemID")
unique_genres = [
    "Action",
    "Thriller",
    "Romance",
    "Western",
    "Children's",
    "Mystery",
    "Fantasy",
    "Film-Noir",
    "Documentary",
    "Comedy",
    "Adventure",
    "Sci-Fi",
    "Horror",
    "Crime",
    "Musical",
    "War",
    "Animation",
    "Drama",
]
for genre in unique_genres:
    movies[genre] = 0
    
for index, row in movies.iterrows():
    genres = row["genres"].split("|")

    for genre in genres:
        movies.at[index, genre] = 1


movies

users = pd.read_csv("../data/ml-100k/u_id_mapping_demographic_.csv", sep="\t")
users = users.sort_values(by="userID")
gender_map = {"M": 0, "F": 1}
users["Gender"] = users["Gender"].map(gender_map)
user_features_numpy = users.to_numpy()
users



In [ ]:

import os
os.chdir("/Users/name/Desktop/work/reranking_fairnes")

from metrics.utils import map_age
users["Age_Code"] = users["Age"].apply(map_age)
users


In [15]:
from mymetrics.GenrePrecisionMulti import GenrePrecisionMulti
from mymetrics.GenreNDCGMulti import GenreNDCGMulti

top_k = 20
###intialize them
gp = GenrePrecisionMulti(users, unique_genres, top_k)
gn = GenreNDCGMulti(users, unique_genres, top_k)


In [ ]:

new_rmatrix = np.zeros(( 943, 20), dtype=int)
for i in range(943):
    new_rmatrix[i]=reco_matrix[i][:20]
new_rmatrix.shape


In [19]:
sens_attr = "Gender"

In [ ]:
gp_plain = gp.compute(new_rmatrix, movies, sens_attr)
gp_plain

In [ ]:
gn_plain = gn.compute(new_rmatrix, movies, sens_attr)
gn_plain

In [ ]:

i=0
xlist=[]
newlist = []
for r in reco_matrix_fair:
    i=i+1
    x=gp.compute(np.array(r), movies, sens_attr)
    xlist.append(x[0])
    newlist.append(x)
    print(f"{x[0]} is the bias score for b={i}")
    


In [ ]:
i=0
xlist=[]
for r in reco_matrix_fair:
    i=i+1
    x=gn.compute(np.array(r), movies, sens_attr)
    xlist.append(x[0])
    print(f"{x[0]} is the bias score for b={i}")


In [ ]:
[gn_plain[0]]+xlist